# Initialisation

## Imports

In [1]:
import sys, os, csv
import logging, importlib

import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

%load_ext autoreload
%autoreload 2

In [3]:
sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC")
from posarmctools.readdata import check_ini, getInfoRecords, checkTimestamps
import posarmctools.ekfnavtools as ekf
import posarmctools.epsgtools as epsg
import posarmctools.sbg as sbg
import posarmctools.posar as posar
from posarmctools.get_notebook_name import get_notebook_name

## Parameters

In [4]:
#ini = "/home/pleroy/DATA/PoSAR-X/PIMA-1/flight1.ini"
ini = "/home/pleroy/DATA/PoSAR-X/PIMA-1/flight_letg_2018_06_27.ini"
refs, (dir_sbg, session, sbg_hours), (dir_posar, day, hours) = check_ini(ini)
root_dir = os.path.join(dir_posar, day)
withPlots = 0
epsg3xxx = epsg.epsg3857

In [39]:
dir_out = os.path.join(dir_posar, "OUT")
try:
    os.makedirs(dir_out, exist_ok = True) 
    print(f"{dir_out} created successfully") 
except OSError as error: 
    print(f"{dir_out} can not be created") 

/home/pleroy/DATA/PoSAR-C/2018_06_27_LETG/2018_06_27/jde/OUT created successfully


## Logger

In [5]:
modulename = get_notebook_name().split('/')[-1].split('.ipynb')[0]
logger = logging.getLogger()
logger.setLevel(logging.INFO)
# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logfile = modulename + ".log"

def loggingSetFh(name, formatter):
    # create file handler which logs even debug messages
    fh = logging.FileHandler(name, mode='w')
    fh.setLevel(logging.DEBUG)
    # set the formatter
    fh.setFormatter(formatter)
    logger.addHandler(fh)

loggingSetFh(logfile, formatter)

## EPSG transformations

In [6]:
if epsg3xxx == epsg.epsg3857:
    epsgStr = "epsg3857"
    
if epsg3xxx == epsg.epsg3948:
    epsgStr = "epsg3948"

## Scene remarkable points

In [7]:
sys.path.insert(0, "/home/pleroy/DATA/PoSAR-X/PIMA-1")

In [8]:
sceneReferencePoints = importlib.import_module(refs.split('.py')[0])
pts = epsg.getReferencePoints(sceneReferencePoints)
ptsLongLat = {pt: pts[pt][::-1] for pt in pts}
ptsEpsg = {pt: epsg.wgs84LongLatToEpsg( ptsLongLat[pt], epsg3xxx ) for pt in pts}
prefixes = epsg.getReferencePointPrefixes(pts)
for prefix in prefixes:
    np.save(f"{prefix}_epsg", 
            [ptsEpsg[pt] for pt in pts if pt.rstrip('0123456789') == prefix]
            )

In [186]:
prefixes

{'J': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]}

In [9]:
from matplotlib.cm import get_cmap

# SBG data

## data path definitions

In [10]:
logLoader = sbg.LogLoader(dir_sbg, session, day, sbg_hours)

##  logGpsPos

### Load data

In [11]:
gps = sbg.Log("sbgLogGpsPos.csv", logLoader, delimiter=",")
Lat, Long, Alt, Undulation, gpsPosStatus = [gps.data[:, gps.names.index(c)]
                                            for c in ('latitude', 'longitude', 'altitude', 'undulation', 'status')]
gpsPosStatus = gpsPosStatus.astype("uint32")
x, y = epsg.wgs84LongLatToEpsg( (Long, Lat), epsg3xxx )

In [12]:
epsg.plotEpsgWithRemarkablePoints(x, y, gps, prefixes, ptsEpsg, epsgStr, day)

In [185]:
prefixes

{'J': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]}

In [13]:
posStatus = np.zeros( gpsPosStatus.shape )
posType = np.zeros( gpsPosStatus.shape )
for idx, val in enumerate(gpsPosStatus):
    posStatus[idx] = val & 0b000011
    posType[idx] = np.right_shift( (val & 0b111111000000), 6)

In [14]:
plt.figure()
plt.subplot(411)
plt.plot(Lat)
plt.subplot(412)
plt.plot(Long)
plt.subplot(413)
plt.plot(Alt)
plt.subplot(414)
plt.plot(Undulation)

In [16]:
fig, ax = plt.subplots(1,1)
ax.plot(gps.timestamps, posStatus, label='posStatus')
ax.plot(gps.timestamps, posType, label='posType')
ax.grid()
ax.legend()

### Plot navigation data jointly with track references

In [17]:
plt.figure()
ax = plt.subplot(111)
ax.plot( Long, Lat )
epsg.addRemarkablePoints(ax, prefixes, ptsLongLat)
plt.title( f"GPS data and track references\n{day} {session} {sbg_hours}" )
plt.grid()
plt.legend()

In [19]:
plt.figure()
for idx, slice_ in enumerate(gps.slices):
    plt.plot( x[slice_], y[slice_], label=hours[idx] )

ax = plt.gca()
epsg.addRemarkablePoints(ax, prefixes, ptsEpsg)

plt.xlabel("x")
plt.ylabel("y")
plt.grid()
plt.legend()
title = f"{epsgStr} {day}"
plt.title(title)
plt.gca().set_aspect('equal')

In [20]:
gps_proj = epsg.wgs84LongLatToEpsg( (Long, Lat), epsg3xxx )
np.save( root_dir + "gps_epsg_transform", gps_proj )

## logGpsVel

In [21]:
vel = sbg.Log("sbgLogGpsVel.dat", logLoader)
velNorth, velEast, velDown, course, gpsVelStatus = [vel.data[:, vel.names.index(c)]
                                                    for c in ('velocityNorth', 'velocityEast', 'velocityDown', 'course', 'status')]
Vel = ( velNorth**2 + velEast**2 + velDown**2) **0.5
gpsVelStatus = gpsVelStatus.astype("uint32")

In [22]:
# STATUS
# 0 SBG_ECOM_VEL_SOL_COMPUTED     A valid solution has been computed.
# 1 SBG_ECOM_VEL_INSUFFICIENT_OBS Not enough valid SV to compute a solution.
# 2 SBG_ECOM_VEL_INTERNAL_ERROR   An internal error has occurred.
# 3 SBG_ECOM_VEL_LIMIT            Velocity limit exceeded.
# TYPE
# 0 SBG_ECOM_VEL_NO_SOLUTION  No valid velocity solution available.
# 1 SBG_ECOM_VEL_UNKNOWN_TYPE An unknown solution type has been computed.
# 2 SBG_ECOM_VEL_DOPPLER      A Doppler velocity has been computed.
# 3 SBG_ECOM_VEL_DIFFERENTIAL A velocity has been computed between two positions.
velStatus = np.zeros( gpsVelStatus.shape )
velType = np.zeros( gpsVelStatus.shape )
for idx, val in enumerate(gpsVelStatus):
    velStatus[idx] = val & 0b000011
    velType[idx] = np.right_shift( (val & 0b111111000000), 6)

In [24]:
fig, ax = plt.subplots(1,1)
ax.plot(vel.timestamps, velStatus, label='velStatus')
ax.plot(vel.timestamps, velType, label='velType')
ax.grid()
ax.legend()

## logUtcData

In [25]:
utc = sbg.LogUtcData("sbgLogUtcData.dat", logLoader)

start ... 9:13:50.750
loop 0 ... 9:26:17.515
loop 1 ... 10:37:52.480
loop 2 ... 11:49:27.450
stop ... 12:45:37.775


## sbgLogEventB

In [26]:
eventB = sbg.LogEventB("sbgLogEventB.dat", logLoader, utc=utc)

In [28]:
fig, ax = plt.subplots(1, 1)
for slice_ in eventB.slices:
    ax.plot(range(slice_.start, slice_.stop) , eventB.data[slice_ ,0], 'x', label="eventB.data")
    ax.plot(range(slice_.start, slice_.stop) , eventB.shiftedTimestamps[slice_ ], '.', label="eventB.shiftedTimestamps")
ax.legend()
ax.grid()

## logEkfEuler

In [29]:
euler = sbg.Log("sbgLogEkfEuler.dat", logLoader)
roll, pitch, yaw = [euler.data[:, euler.names.index(c)] for c in ('roll', 'pitch', 'yaw')]

## logEkfNav

In [30]:
nav = sbg.Log("sbgLogEkfNav.dat", logLoader)
navLat, navLong, navAlt, navUndulation, navStatus = [nav.data[:, nav.names.index(c)]
                                                     for c in ('Lat', 'Long', 'Alt', 'undulation', 'status')]
nav_status = navStatus.astype("uint32")

In [31]:
solution_mode = np.zeros( nav_status.shape )
for idx, val in enumerate(nav_status):
    solution_mode[idx] = val & 0b1111

In [35]:
fig, ax1 = plt.subplots(1,1)

color = 'tab:blue'
ax1.plot(nav.timestamps, solution_mode,color=color)
ax1.set_ylabel('sbg_ecom_solution_mode', color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
color = 'tab:red'
ax2.plot(nav.timestamps, navAlt, color=color)
ax2.set_ylabel('navAlt', color=color)
ax2.tick_params(axis='y', labelcolor=color)
ax2.plot(gps.timestamps, Alt, color='tab:green')
ax2.grid()

In [40]:
nav_proj = epsg.wgs84LongLatToEpsg((navLong, navLat), epsg3xxx)
np.save(os.path.join(dir_out, "nav_epsg_transform"), nav_proj)

In [41]:
plt.figure()
plt.plot( nav_proj[0], nav_proj[1], label = "nav" )
plt.plot( gps_proj[0], gps_proj[1], label = "gps" )
ax = plt.gca()
epsg.addRemarkablePoints(ax, prefixes, ptsEpsg)
plt.legend()
plt.grid()
plt.gca().set_aspect('equal')
title = "comparison between nav and gps data"
plt.title(title)

Text(0.5, 1.0, 'comparison between nav and gps data')

/home/pleroy/anaconda3/lib/python3.7/site-packages/ipykernel/eventloops.py:106: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  app.exec_()


In [42]:
plt.figure()
plt.plot(nav.timestamps, navAlt, label = "nav")
plt.plot(gps.timestamps, Alt, '.', label = "gps")
plt.legend()
plt.grid()

/home/pleroy/anaconda3/lib/python3.7/site-packages/ipykernel/eventloops.py:106: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  app.exec_()


# PoSAR-MC data

In [43]:
expectedPeriod = 900000
args_record = 0

In [170]:
loggingSetFh(logfile, formatter)

In [173]:
rec_dates = [ f"{day}_" + h for h in hours ]
rec_dirs = [os.path.join(dir_posar, d) for d in rec_dates]
records = [posar.Record(rec_dir, "record", "bin", utc=utc, period=expectedPeriod, version="C_v2") for rec_dir in rec_dirs]

rec_dir /home/pleroy/DATA/PoSAR-C/2018_06_27_LETG/2018_06_27/jde/2018_06_27_12_39_39
rec_dir /home/pleroy/DATA/PoSAR-C/2018_06_27_LETG/2018_06_27/jde/2018_06_27_12_45_51
rec_dir /home/pleroy/DATA/PoSAR-C/2018_06_27_LETG/2018_06_27/jde/2018_06_27_12_52_49
rec_dir /home/pleroy/DATA/PoSAR-C/2018_06_27_LETG/2018_06_27/jde/2018_06_27_12_59_03
rec_dir /home/pleroy/DATA/PoSAR-C/2018_06_27_LETG/2018_06_27/jde/2018_06_27_13_05_40
rec_dir /home/pleroy/DATA/PoSAR-C/2018_06_27_LETG/2018_06_27/jde/2018_06_27_13_11_42
rec_dir /home/pleroy/DATA/PoSAR-C/2018_06_27_LETG/2018_06_27/jde/2018_06_27_13_18_13
rec_dir /home/pleroy/DATA/PoSAR-C/2018_06_27_LETG/2018_06_27/jde/2018_06_27_13_24_11
rec_dir /home/pleroy/DATA/PoSAR-C/2018_06_27_LETG/2018_06_27/jde/2018_06_27_13_30_53
rec_dir /home/pleroy/DATA/PoSAR-C/2018_06_27_LETG/2018_06_27/jde/2018_06_27_13_36_46
rec_dir /home/pleroy/DATA/PoSAR-C/2018_06_27_LETG/2018_06_27/jde/2018_06_27_13_43_13
rec_dir /home/pleroy/DATA/PoSAR-C/2018_06_27_LETG/2018_06_27/jde/

In [174]:
records[2].out_dir

'/home/pleroy/DATA/PoSAR-C/2018_06_27_LETG/2018_06_27/jde/OUT/2018_06_27_12_52_49'

In [126]:
validFiles = [record.day_hour for idx, record in enumerate(records) if record.idxTime.size == 0]
notValidFiles = [record.day_hour for idx, record in enumerate(records) if record.idxTime.size > 0]

In [127]:
print(f"*** VALID FILES ***\n{validFiles}\n*** NOT VALID FILES ***\n{notValidFiles})")

*** VALID FILES ***
['2018_06_27_12_45_51', '2018_06_27_12_52_49', '2018_06_27_12_59_03', '2018_06_27_13_11_42', '2018_06_27_13_18_13', '2018_06_27_13_24_11', '2018_06_27_13_30_53', '2018_06_27_13_36_46']
*** NOT VALID FILES ***
['2018_06_27_12_39_39', '2018_06_27_13_05_40', '2018_06_27_13_43_13', '2018_06_27_13_49_07'])


In [194]:
fig, (ax, ax1) = plt.subplots(2, 1, sharex=True)

for index, record in enumerate(records):
    ax.plot(record.shiftedLogEvents, record.shiftedLogEvents, '.', label=f"{record.day_hour}")
ax.vlines(utc.timestamps[utc.loopbacks[1:-1]], 0, 2**32 * (len(utc.loopbacks) - 2 + 0.5))
ax.legend()
ax.grid()

for num, slice_ in enumerate(eventB.slices):
    ax1.plot(eventB.shiftedTimestamps[slice_], eventB.shiftedTimestamps[slice_], '.', label=eventB.loader.hours[num])
ax1.vlines(utc.timestamps[utc.loopbacks[1:-1]], 0, 2**32 * (len(utc.loopbacks) - 2 + 0.5))
ax1.legend()
ax1.grid()
fig.suptitle("loopbacks during the records")

Text(0.5, 0.98, 'loopbacks during the records')

In [193]:
fig, (ax, ax1) = plt.subplots(2,1, sharex=True)

ax.plot(record.modificationTimestamps, 's',
        markeredgecolor='black',
        markerfacecolor='white',
        label=f"modificationTimestamps ({len(record.modificationTimestamps)})")
ax.plot(record.logEvents, 'x',
        markeredgecolor='black',
        markerfacecolor='white',
        label=f"posarTimestamps ({len(record.logEvents)})")
delta = record.logEvents[0] - record.modificationTimestamps[0]
ax.plot(record.logEvents - delta, '.',
        markeredgecolor='black',
        markerfacecolor='white',
        label=f"posarTimestamps with constant shift ({len(record.logEvents)})")
ax.legend()
ax.grid()
ax.set_title(record.day_hour)

diff = np.diff(record.modificationTimestamps)
diff1 = np.diff(record.logEvents)
diff2 = np.diff(record.modificationSeconds) * 1e6
ax1.plot(diff, 's',
         markeredgecolor='black',
         markerfacecolor='white',
         label="diff modificationTimestamps")
ax1.plot(diff1, 'x',
         markeredgecolor='black',
         markerfacecolor='white',
         label="diff posarTimestamps")
ax1.plot(diff2, 'o',
         markeredgecolor='black',
         markerfacecolor='yellow',
         label="diff modificationSeconds")
ax1.grid()
ax1.legend()

In [192]:
fig, ax = plt.subplots(1, 1)
toPlot = np.array(record.modificationSeconds) * 1e6
ax.plot(toPlot, np.ones(len(toPlot)), 'x',
        markeredgecolor='black',
        markerfacecolor='white',
        label=f"modificationSeconds ({len(record.modificationTimestamps)})")
ax.grid()
ax.legend()

## Selected record

In [181]:
record = records[2]

In [97]:
t0 = record.shiftedLogEvents[0]
t1 = record.shiftedLogEvents[-1]
delta = 10e6
idx = np.where((eventB.shiftedTimestamps>t0-delta) & (eventB.shiftedTimestamps<t1+delta))

fig, (ax, ax1) = plt.subplots(2, 1, sharex=True)

ax.plot(record.shiftedLogEvents/1e6, record.shiftedLogEvents/1e6,
        "o", markeredgecolor="k", color="w", label=f"PoSAR LogEventB {record.day_hour}")
ax.plot(eventB.shiftedTimestamps[idx]/1e6, eventB.shiftedTimestamps[idx]/1e6,
        '.r', label="SBG LogEventB shiftedTimestamps")
ax.legend()
ax.grid()

ax1.plot(record.shiftedLogEvents[1:]/1e6, np.diff(record.shiftedLogEvents),
         "o", markeredgecolor="k", color="w", label=f"[diff] PoSAR LogEventB {record.day_hour}")
ax1.plot(eventB.shiftedTimestamps[idx][1:]/1e6, np.diff(eventB.shiftedTimestamps[idx]), 
         '.r', label="[diff] SBG LogEventB shiftedTimestamps")
ax1.grid()
ax1.legend()

fig.suptitle("loopbacks during the records")

Text(0.5, 0.98, 'loopbacks during the records')

### Read parameters

In [99]:
samplesPerRamp = record.parameters.samplesPerRamp
rampsPerFile = record.parameters.rampsPerFile
rampsPerBuffer = record.parameters.rampsPerBuffer
samplesPerFile = record.parameters.samplesPerRamp * params.rampsPerFile
buffersPerFile = record.parameters.buffersPerFile
T_files = samplesPerFile / 10e6

## Data interpolation

In [117]:
def getInterpolatedGps(x, xp, Lat, Long, Alt):
    Lat_interp = np.interp(x, xp, Lat)
    Long_interp  = np.interp(x, xp, Long)
    Alt_interp  = np.interp(x, xp, Alt)
    return Lat_interp, Long_interp, Alt_interp

def getInterpolatedVelCourse(x, xp, Vel, course):
    Vel_record = np.interp(x, xp, Vel)
    course_record  = np.interp(x, xp, course)
    return Vel_record, course_record

def getInterpolatedVel(x, xp, North, East, Down):
    velNorth_records = np.interp(x, xp, North)
    velEast_records  = np.interp(x, xp, East)
    velDown_records  = np.interp(x, xp, Down)
    return velNorth_records, velEast_records, velDown_records

### logEkfEuler

In [119]:
roll_selection = np.interp(record.shiftedLogEvents, euler.timestamps, roll)
pitch_selection = np.interp(record.shiftedLogEvents, euler.timestamps, pitch)
yaw_selection = np.interp(record.shiftedLogEvents, euler.timestamps, yaw)

### logGpsPos

In [120]:
Lat_Long_Alt = getInterpolatedGps(record.shiftedLogEvents, gps.timestamps, Lat, Long, Alt)
selectedEpsg = epsg.wgs84LongLatToEpsg((Lat_Long_Alt[1], Lat_Long_Alt[0]), epsg3xxx)

### logGpsVel

In [121]:
Vel_course = getInterpolatedVelCourse(record.shiftedLogEvents, vel.timestamps, Vel, course)
Vel_records = getInterpolatedVel(record.shiftedLogEvents, vel.timestamps, velNorth, velEast, velDown) 

## Plot navigation data and record periods

In [151]:
fig, ax = plt.subplots(1,1)

for index, slice_ in enumerate(gps.slices):
    ax.plot( x[slice_], y[slice_], label=gps.loader.hours[index] )
    
epsg.addRemarkablePoints(ax, prefixes, ptsEpsg)
    
ax.plot( selectedEpsg[0], selectedEpsg[1], ".r", markeredgecolor='black', label=record.day_hour )
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.grid()
ax.legend()
title = epsgStr + " " + day
ax.set_title(title)
ax.set_aspect('equal')
fig.savefig( f"{record.out_dir}/{title}.png", bbox_inches='tight')

In [152]:
fig, ax = plt.subplots(1,1)
ax.plot( Lat_Long_Alt[1], Lat_Long_Alt[0], ".r", markeredgecolor='black', 
         label=record.day_hour )
title = f"record locations-{record.day_hour}"
ax.set_title(title)
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.legend()
ax.grid()
fig.savefig( f"{record.out_dir}/{title}.png", bbox_inches='tight')

In [153]:
fig, ax = plt.subplots(1,1)
ax.plot( gps.timestamps, Alt )
ax.plot( record.shiftedLogEvents, Lat_Long_Alt[2], "yD", markeredgecolor = 'black' )
ax.grid()
title = "Altitude - " + record.day_hour + " - files {} to {}".format(record.binNums[0], record.binNums[-1])
ax.set_title( title )
fig.savefig( f"{record.out_dir}/{title}.png", bbox_inches='tight')

In [154]:
files = "files {} to {}".format(record.binNums[0], record.binNums[-1])

In [159]:
plt.figure()

title = "altitude - velocity - course " + record.day_hour

plt.subplot(311)
plt.title( title )
plt.plot(record.shiftedLogEvents, Lat_Long_Alt[2], 'Dy', markeredgecolor = 'k', label="altitude " + files)
plt.legend()
plt.grid()

plt.subplot(312)
plt.plot(record.shiftedLogEvents, Vel_course[0], 'Dy', markeredgecolor = 'k', label="velocity " + files)
plt.legend()
plt.grid()

courseInDeg = Vel_course[1]
courseInRad = Vel_course[1] / 180 * np.pi
courseUnWrapped = np.unwrap(courseInRad, discont=np.pi) * 180 / np.pi
plt.subplot(313)
plt.plot(record.shiftedLogEvents, courseInDeg, 'Dy', markeredgecolor = 'k', label="course " + files)
plt.legend()
plt.grid()

plt.savefig(record.out_dir + "/" + title + ".png", bbox_inches='tight')

In [183]:
plt.figure()

title = "pitch - roll - yaw " + record.day_hour

plt.subplot(311)
plt.title( title )
plt.plot(record.shiftedLogEvents, pitch_selection * 180 / np.pi, 'Dy', markeredgecolor = 'k', label="pitch [°] " + files)
plt.legend()
plt.grid()

plt.subplot(312)
plt.plot(record.shiftedLogEvents, roll_selection * 180 / np.pi, 'Dy', markeredgecolor = 'k',  label="roll [°] " + files)
plt.legend()
plt.grid()

plt.subplot(313)
plt.plot(record.shiftedLogEvents, np.unwrap(yaw_selection) * 180 / np.pi, 'Dy', markeredgecolor = 'k',  label="yaw [°] " + files)
plt.legend()
plt.grid()

plt.savefig(record.out_dir + "/" + title + ".png", bbox_inches='tight')

In [184]:
plt.figure(figsize=(8, 6))

title = "north - east - down " + record.day_hour

plt.subplot(311)
plt.title( title )
plt.plot(record.shiftedLogEvents, Vel_records[0], 'Dy', markeredgecolor = 'k', label="north [m/s] " + files)
plt.legend()
plt.grid()

plt.subplot(312)
plt.plot(record.shiftedLogEvents, Vel_records[1], 'Dy', markeredgecolor = 'k',  label="east [m/s] " + files)
plt.legend()
plt.grid()

plt.subplot(313)
plt.plot(record.shiftedLogEvents, Vel_records[2], 'Dy', markeredgecolor = 'k',  label="down [m/s] " + files)
plt.legend()
plt.grid()

plt.savefig(record.out_dir + "/" + title + ".png", bbox_inches='tight')

In [179]:
record.out_dir

'/home/pleroy/DATA/PoSAR-C/2018_06_27_LETG/2018_06_27/jde/2018_06_27_12_52_49'

# Track approximation

## Orientation vector

In [ ]:
# East-West s0  => s1  14_12_21
# West-East s2  => s3  14_14_43
# South-North s4 => s5 14_18_28
# North-South s6 => s7 14_21_00
xA, yA = s4_epsg
xB, yB = s5_epsg
JA = s4
JB = s5

In [ ]:
xAB = (xB - xA) / ( (xB - xA)**2 + (yB - yA)**2 )**0.5
yAB = (yB - yA) / ( (xB - xA)**2 + (yB - yA)**2 )**0.5

In [ ]:
# unit vectors
ux = ( xAB, yAB )
uy = ( -yAB, xAB )

In [ ]:
# build sceneReferencePoint
JA_epsg = epsg.wgs84ToEpsg( (JA[1], JA[0]), epsg3xxx )
JB_epsg = epsg.wgs84ToEpsg( (JB[1], JB[0]), epsg3xxx )
JAJB = ( (JA_epsg[0]+JB_epsg[0])/2,
        (JA_epsg[1]+JB_epsg[1])/2 )
shiftInY = 0
sceneReferencePoint = JAJB[0] + shiftInY * uy[0], JAJB[1] + shiftInY * uy[1]

In [ ]:
import json

In [ ]:
track_model = { "trackModel" : "model(x, p) = p[1]*x + p[2]",
               "ux" : (ux[0], ux[1]),
               "uy" : (uy[0], uy[1]),
               "origX" : sceneReferencePoint[0],
               "origY" : sceneReferencePoint[1]
              }

In [ ]:
track_model

In [ ]:
trackFilename = data_dir[selectedRecord] + "/track_model.json"
with open( trackFilename, 'w' ) as f:
    json.dump( track_model, f )

In [ ]:
x_selection = np.interp( selectedTimeStamp, gpsNewTimestamps, gps_proj[0] )
y_selection = np.interp( selectedTimeStamp, gpsNewTimestamps, gps_proj[1] )
np.save( data_dir[selectedRecord] + "/track_selection_proj", np.stack( (x_selection, y_selection ), -1 ) )

In [ ]:
plt.figure()

plt.plot( gps_proj[0], gps_proj[1], "gray" )

plt.plot( x_selection, y_selection, 'Dy', markerEdgecolor='k',
        label = "files {} to {} ".format( firstRecord, nbRecords[selectedRecord] ) + data_date[selectedRecord] )

ax = plt.gca()
ax.set_aspect("equal")
plt.xlabel("location of the records in the scene coordinate system\n")
plt.grid()
plt.legend()
ax = plt.gca()

In [ ]:
tmpJA = epsg.wgs84ToEpsg( (JA[1], JA[0]), epsg3xxx )
tmpJB = epsg.wgs84ToEpsg( (JB[1], JB[0]), epsg3xxx )
                      
plt.figure()
plt.plot(gps_proj[0], gps_proj[1])
plt.plot(tmpJA[0], tmpJA[1], 'o', markeredgecolor='k', label="JA")
plt.plot(tmpJB[0], tmpJB[1], 'o', markeredgecolor='k', label="JB")
plt.plot(JAJB[0], JAJB[1], 'o', markeredgecolor='k', label="reference track center")
plt.plot(sceneReferencePoint[0], sceneReferencePoint[1], 'o', markeredgecolor='k', label="scene reference point")
plt.gca().set_aspect("equal")
plt.grid()
plt.legend()
title = "transformation from WGS84 to " + epsgStr + "\n" + data_dir[selectedRecord]
plt.title(title)
ax = plt.gca()

# Interpolate navigation data for all ramps

In [ ]:
# rampNumber timeStamp x y z
nbRecordToAdd = 10

rampNumber = np.arange( rampsPerFile * ( nbRecords[selectedRecord] + nbRecordToAdd ) )

rampNumber_ext = np.arange(nbRecords[selectedRecord] + nbRecordToAdd) * rampsPerFile

lastDelta = selectedTimeStamp[-1] - selectedTimeStamp[-2]
# there is a difference between the number of valid timestamps and the number of records
diffValidTimestampsNbRecords = selectedTimeStamp.size - nbRecords[selectedRecord]
addedTimestamps = np.arange( 1, nbRecordToAdd - diffValidTimestampsNbRecords + 1 ) * lastDelta
print("lastDelta = {}".format(lastDelta) )
timeStamp_ext_a = np.concatenate( (selectedTimeStamp, 
                                 selectedTimeStamp[-1] + addedTimestamps ) )

In [ ]:
timestampsAllRamps = np.interp( rampNumber, rampNumber_ext, timeStamp_ext_a)

## logGpsPos (a)
Two options: numpy.interp vs scipy.interpolate.interp1d (with kind='linear')

### xyz

In [ ]:
gps_x = np.interp( timestampsAllRamps, gpsNewTimestamps, gps_proj[0] )
gps_y = np.interp( timestampsAllRamps, gpsNewTimestamps, gps_proj[1] )
gps_z = np.interp( timestampsAllRamps, gpsNewTimestamps, Alt )
xyz_proj_allRamps = np.stack( (rampNumber, timestampsAllRamps, 
                               gps_x, gps_y, gps_z), -1 )

In [ ]:
np.save( data_dir[selectedRecord] + "/rampNumber_timeStamp_xyz_gps_a", xyz_proj_allRamps )

In [ ]:
plt.figure()
plt.plot( gps_x, gps_y, '.b', label = 'ramps' )
plt.plot( x_selection, y_selection, 
         'D', markerfacecolor='None', markeredgecolor='red',
         label = "files {} to {}".format( firstRecord, nbRecords[selectedRecord] ) )
plt.legend()
plt.title("xy")
plt.grid()

### lla latitude longitude altitude

In [ ]:
allRamps_lat = np.interp( timestampsAllRamps, gpsNewTimestamps, Lat )
allRamps_long = np.interp( timestampsAllRamps, gpsNewTimestamps, Long )
allRamps_alt = np.interp( timestampsAllRamps, gpsNewTimestamps, Alt )
latLongAlt_allRamps = np.stack( (rampNumber, timestampsAllRamps,
                                 allRamps_lat, allRamps_long, allRamps_alt), -1 )

In [ ]:
np.save( data_dir[selectedRecord] + "/rampNumber_timeStamp_latLongAlt_gps_a", latLongAlt_allRamps )

## logEkfNav (a)

In [ ]:
nav_x = np.interp( timestampsAllRamps, navNewTimestamps, nav_proj[0] )
nav_y = np.interp( timestampsAllRamps, navNewTimestamps, nav_proj[1] )
nav_z = np.interp( timestampsAllRamps, navNewTimestamps, nav_Alt )
nav_u = np.interp( timestampsAllRamps, navNewTimestamps, nav_Undulation )
nav_xyz_allRamps = np.stack( (rampNumber, timestampsAllRamps, 
                              nav_x, nav_y, nav_z), -1 )

In [ ]:
np.save( data_dir[selectedRecord] + "/rampNumber_timeStamp_xyz_nav_a", nav_xyz_allRamps )

In [ ]:
nav_allRamps_lat = np.interp( timestampsAllRamps, navNewTimestamps, nav_Lat )
nav_allRamps_long = np.interp( timestampsAllRamps, navNewTimestamps, nav_Long )
nav_allRamps_alt = np.interp( timestampsAllRamps, navNewTimestamps, nav_Alt )
nav_latLongAlt_allRamps = np.stack( (rampNumber, timestampsAllRamps, 
                              nav_allRamps_lat, nav_allRamps_long, nav_allRamps_alt), -1 )

In [ ]:
np.save( data_dir[selectedRecord] + "/rampNumber_timeStamp_latLongAlt_nav_a", nav_latLongAlt_allRamps )

## Compare gps and nav

In [ ]:
plt.figure()
plt.plot( gps_x, gps_y, label="gps" )
plt.plot( nav_x, nav_y, label="nav" )
plt.legend()
plt.grid()
plt.title("x-y")

In [ ]:
plt.figure()
plt.plot( gps_z, label="gps"  )
plt.plot( nav_z, label="nav" )
plt.legend()
plt.grid()
plt.title("z")

## logGpsPos (b)

In [ ]:
# rampNumber timeStamp x y z

timeStamp_ext_b = timeStamp_ext_a + params.rampPeriod / 2

timeStamp = np.interp( rampNumber, rampNumber_ext, timeStamp_ext_b)
x = np.interp( timeStamp, gpsNewTimestamps, gps_proj[0] )
y = np.interp( timeStamp, gpsNewTimestamps, gps_proj[1] )
z = np.interp( timeStamp, gpsNewTimestamps, Alt )
xyz_proj_allRamps = np.stack( (rampNumber, timeStamp, x, y, z), -1 )

In [ ]:
np.save( data_dir[selectedRecord] + "/rampNumber_timeStamp_xyz_b", xyz_proj_allRamps )